### Generate M random potentials on [0, L] as sum of n gaussian kernels on G discretization points


there are 3 uniformly distributed random parameters a, b, c
the potential is calculated by:

$$
V(x) = -\sum_{n=1}^{3} a_n \exp \left(-\frac{(x - b_n)^2}{2c_n^2} \right)
$$

### Then solve the schroedinger equation in atomic units for 4 spinless non-interacting fermions.

The shooting numerov algorithm calculates a forward pass and a backward pass solution.

The final solution is then found by aligning the two passes and finding a good split point to concatenate them.

All integrals are approximated by the trapezoidal method.
The numerical differentiation for the laplacian uses the standard approximation.

### Usage:

Setting the dataset variable to different values will allow generating all the variations of the datasets:
standard, recreate, recreate/fine_0.125, ..., recreate/fine_10


In [ ]:
notebook_path = "Projects/QuantumFlow/notebooks"
try:
    import os
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir("/content/gdrive/My Drive/" + notebook_path)
except:
    pass

%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as widgets
from IPython.display import display

import sys
sys.path.append('../')

from quantumflow.generate_datasets import generate_potentials, generate_datasets
from quantumflow.colab_utils import test_colab_devices

has_gpu, has_tpu = test_colab_devices()
if has_gpu: print("Found GPU")
if has_tpu: print("Found TPU")

!pip install -q ruamel.yaml
data_dir = "../data"

## Recreate

In [ ]:
experiment = "recreate"
base_dir = os.path.join(data_dir, experiment)
if not os.path.exists(base_dir): os.makedirs(base_dir)
file_hyperparams = os.path.join(base_dir, "hyperparams.config")

In [ ]:
%%writefile $file_hyperparams
globals: []

train: &DEFAULT
    filename: dataset_train
    seed: 0
    n_orbitals: 4 # N
    dataset_size: 100 # M

    discretisation_points: 500 # G
    interval_length: 1.0 # [Bohr]
    n_gauss: 3 # number of gaussians 
    n_method: sum # sum or mean the gaussians
    a_minmax: [1.0, 10.0] # [hartree] 
    b_minmax: [0.4, 0.6] # nomalized x
    c_minmax: [0.03, 0.1] # nomalized x

    dtype: double
    numerov_init_slope: 5.0

validate:
    <<: *DEFAULT
    filename: dataset_validate
    seed: -1
    dataset_size: 800

test:
    <<: *DEFAULT
    filename: dataset_test
    seed: -2
    dataset_size: 8000

fine:
    <<: *DEFAULT
    filename: dataset_fine
    discretisation_points: 4991

float:
    <<: *DEFAULT
    filename: dataset_float
    dtype: float

In [ ]:
generate_datasets(data_dir=data_dir, experiment='recreate', generate_names=['train', 'validate', 'test', 'fine', 'float'])

##Diverse

In [ ]:
experiment = "diverse"
base_dir = os.path.join(data_dir, experiment)
if not os.path.exists(base_dir): os.makedirs(base_dir)
file_hyperparams = os.path.join(base_dir, "hyperparams.config")

In [ ]:
%%writefile $file_hyperparams
globals: []

train: &DEFAULT
    filename: dataset_train
    seed: 0
    n_orbitals: 4
    dataset_size: 8000

    discretisation_points: 1001
    interval_length: 20.0
    n_gauss: 5
    n_method: mean
    a_minmax: [0.0, 50.0]
    b_minmax: [0.3, 0.7]
    c_minmax: [0.02, 0.06]

    dtype: double
    numerov_init_slope: 1e-128

validate:
    <<: *DEFAULT
    filename: dataset_validate
    seed: -1
    dataset_size: 800

test:
    <<: *DEFAULT
    filename: dataset_test
    seed: -2

fine:
    <<: *DEFAULT
    filename: dataset_fine
    discretisation_points: 4991

float:
    <<: *DEFAULT
    filename: dataset_float
    dtype: float
    numerov_init_slope: 1e-5

In [ ]:
generate_datasets(data_dir=data_dir, experiment='diverse', generate_names=['train', 'validate', 'test', 'float'])